In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

#MNIST With CNN
이번 노트북에서는 이미 다뤄본 적이 있는 예제인 MNIST 데이터셋을 CNN을 통해 학습하고 2장에서 얻었던 결과와 비교해보겠습니다. 

MNIST 데이터셋에 대해서는 앞에서 다룬 바가 있으므로 자세한 설명은 생략하겠습니다.


MNIST의 데이터는 28 X 28의 이미지 이므로 input_shape에 (28, 28, 1) 을 넣어줍니다.

처음에는 Convolution 부분만 넣어보겠습니다.

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

W0810 18:05:33.445394 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 18:05:33.491284 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 18:05:33.509223 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 18:05:33.546124 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Convolution 구조를 출력해보겠습니다.

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


현재의 출력은 3D로 이루어지지만 
우리가 원하는 값은 0~9까지의 숫자로 이루어져야 합니다.
3D 출력을 1D 텐서로 펼치는 작업을 합니다.

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Output이 10개로 나누어지므로 10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 Softmax 함수를 사용해보겠습니다.

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

요약을 보면 도중에 Convolution 후에 576크기의 벡터로 펼쳐지고, 이후 Dense층을 거치면서 10개의 값으로 나뉘는 것을 볼 수 있습니다.

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

5회의 에포크, batch_size는 64로 학습을 시작하겠습니다.

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

W0810 18:05:34.239271 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0810 18:05:34.263236 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0810 18:05:34.391884 18692 deprecation.py:323] From C:\Users\yhkim\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0810 18:05:34.581396 18692 deprecation_wrapper.py:119] From C:\Users\yhkim\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v

Epoch 1/5
60000/60000 [==============================] - 32s 541us/step - loss: 0.1825 - acc: 0.9423
Epoch 2/5
60000/60000 [==============================] - 32s 529us/step - loss: 0.0472 - acc: 0.9854
Epoch 3/5
60000/60000 [==============================] - 32s 527us/step - loss: 0.0323 - acc: 0.9898
Epoch 4/5
60000/60000 [==============================] - 32s 531us/step - loss: 0.0244 - acc: 0.9919
Epoch 5/5
60000/60000 [==============================] - 35s 576us/step - loss: 0.0199 - acc: 0.9940


10000개의 테스트 셋을 통해 정확도를 판단하겠습니다.

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 2s 168us/step


In [9]:
test_acc

0.9911

#테스트 결과 약 99%의 정확도를 얻었습니다.
2장에서 MLP를 사용했을 때 97.8%의 정확도가 나왔던것을 생각할 때 기존의 방식보다 CNN을 사용했을때 정확도가 상승했음을 알 수 있습니다.